In [1]:
# -*- coding: utf-8 -*-
'''An implementation of sequence to sequence learning for performing addition

Input: "535+61"
Output: "596"
Padding is handled by using a repeated sentinel character (space)

Input may optionally be inverted, shown to increase performance in many tasks in:
"Learning to Execute"
http://arxiv.org/abs/1410.4615
and
"Sequence to Sequence Learning with Neural Networks"
http://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf
Theoretically it introduces shorter term dependencies between source and target.

Two digits inverted:
+ One layer LSTM (128 HN), 5k training examples = 99% train/test accuracy in 55 epochs

Three digits inverted:
+ One layer LSTM (128 HN), 50k training examples = 99% train/test accuracy in 100 epochs

Four digits inverted:
+ One layer LSTM (128 HN), 400k training examples = 99% train/test accuracy in 20 epochs

Five digits inverted:
+ One layer LSTM (128 HN), 550k training examples = 99% train/test accuracy in 30 epochs
'''

from __future__ import print_function
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range

/Users/f2bcf/anaconda/envs/SNF-env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.

        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One hot encode given string C.

        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)

In [3]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [4]:
# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 3
INVERT = True

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

# All the numbers, plus sign and space for padding.
chars = '0123456789+ '
ctable = CharacterTable(chars)

In [5]:
questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789'))
                    for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    # Answers can be of maximum size DIGITS + 1.
    ans += ' ' * (DIGITS + 1 - len(ans))
    if INVERT:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 50000


In [6]:
# Print example test data.
# Note that questions are inverted in the training for better performance.
for i in range(10):
    print('%s = %s' %(questions[i][::-1],expected[i]))

82+4    = 86  
22+8    = 30  
3+43    = 46  
18+8    = 26  
85+7    = 92  
828+36  = 864 
48+5    = 53  
32+3    = 35  
195+3   = 198 
2+35    = 37  


In [7]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

Vectorization...


In [8]:
# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

In [9]:
# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

In [10]:
print('Training Data:')
print(x_train.shape)
print(y_train.shape)

Training Data:
(45000, 7, 12)
(45000, 4, 12)


In [11]:
print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

Validation Data:
(5000, 7, 12)
(5000, 4, 12)


In [12]:
# Try replacing GRU, or SimpleRNN.
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

In [13]:
print('Build model...')
model = Sequential()
# "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
# As the decoder RNN's input, repeatedly provide with the last hidden state of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
model.add(layers.RepeatVector(DIGITS + 1))
# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(LAYERS):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

Build model...


In [14]:
# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 12)             1548      
_________________________________________________________________
activation_1 (Activation)    (None, 4, 12)             0         
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


In [18]:
no_iter=200

In [19]:
# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, no_iter):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if INVERT else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 25s 558us/step - loss: 0.7537 - acc: 0.7351 - val_loss: 0.7524 - val_acc: 0.7332
Q 896+208 T 1104 ☒ 1100
Q 305+61  T 366  ☒ 367 
Q 8+607   T 615  ☒ 613 
Q 785+71  T 856  ☒ 850 
Q 64+881  T 945  ☒ 941 
Q 411+3   T 414  ☒ 415 
Q 12+216  T 228  ☒ 222 
Q 297+319 T 616  ☑ 616 
Q 881+864 T 1745 ☒ 1749
Q 10+257  T 267  ☒ 260 

--------------------------------------------------
Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 25s 561us/step - loss: 0.7131 - acc: 0.7495 - val_loss: 0.7382 - val_acc: 0.7246
Q 832+371 T 1203 ☒ 1200
Q 603+0   T 603  ☒ 605 
Q 902+2   T 904  ☒ 903 
Q 211+48  T 259  ☒ 261 
Q 627+52  T 679  ☒ 683 
Q 8+188   T 196  ☒ 195 
Q 14+90   T 104  ☒ 105 
Q 89+1    T 90   ☒ 80  
Q 71+44   T 115  ☑ 115 
Q 870+98  T 968  ☒ 966 

-----------------

45000/45000 [==============================] - 25s 556us/step - loss: 0.0233 - acc: 0.9979 - val_loss: 0.0275 - val_acc: 0.9961
Q 91+198  T 289  ☑ 289 
Q 556+46  T 602  ☑ 602 
Q 108+458 T 566  ☑ 566 
Q 757+691 T 1448 ☑ 1448
Q 18+57   T 75   ☑ 75  
Q 81+374  T 455  ☑ 455 
Q 3+717   T 720  ☑ 720 
Q 88+7    T 95   ☑ 95  
Q 78+737  T 815  ☑ 815 
Q 487+757 T 1244 ☑ 1244

--------------------------------------------------
Iteration 16
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 25s 555us/step - loss: 0.0268 - acc: 0.9956 - val_loss: 0.0469 - val_acc: 0.9877
Q 307+1   T 308  ☑ 308 
Q 482+245 T 727  ☑ 727 
Q 182+268 T 450  ☑ 450 
Q 803+91  T 894  ☑ 894 
Q 761+591 T 1352 ☑ 1352
Q 662+452 T 1114 ☑ 1114
Q 0+398   T 398  ☒ 397 
Q 30+3    T 33   ☒ 34  
Q 207+517 T 724  ☑ 724 
Q 923+828 T 1751 ☑ 1751

--------------------------------------------------
Iteration 17
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [===

45000/45000 [==============================] - 25s 558us/step - loss: 0.0038 - acc: 0.9998 - val_loss: 0.0063 - val_acc: 0.9989
Q 739+24  T 763  ☑ 763 
Q 8+39    T 47   ☑ 47  
Q 804+497 T 1301 ☑ 1301
Q 544+754 T 1298 ☑ 1298
Q 82+839  T 921  ☑ 921 
Q 632+838 T 1470 ☑ 1470
Q 281+987 T 1268 ☑ 1268
Q 220+3   T 223  ☑ 223 
Q 23+221  T 244  ☑ 244 
Q 275+643 T 918  ☑ 918 

--------------------------------------------------
Iteration 30
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 25s 554us/step - loss: 0.0034 - acc: 0.9998 - val_loss: 0.0065 - val_acc: 0.9988
Q 653+57  T 710  ☑ 710 
Q 412+211 T 623  ☑ 623 
Q 27+19   T 46   ☑ 46  
Q 924+4   T 928  ☑ 928 
Q 8+114   T 122  ☑ 122 
Q 232+794 T 1026 ☑ 1026
Q 545+7   T 552  ☑ 552 
Q 67+376  T 443  ☑ 443 
Q 46+438  T 484  ☑ 484 
Q 77+150  T 227  ☑ 227 

--------------------------------------------------
Iteration 31
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [===

45000/45000 [==============================] - 25s 549us/step - loss: 0.0019 - acc: 0.9998 - val_loss: 0.0039 - val_acc: 0.9991
Q 49+857  T 906  ☑ 906 
Q 498+821 T 1319 ☑ 1319
Q 488+261 T 749  ☑ 749 
Q 373+391 T 764  ☑ 764 
Q 11+39   T 50   ☑ 50  
Q 98+133  T 231  ☑ 231 
Q 3+245   T 248  ☑ 248 
Q 22+45   T 67   ☑ 67  
Q 664+387 T 1051 ☑ 1051
Q 379+20  T 399  ☑ 399 

--------------------------------------------------
Iteration 44
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 25s 550us/step - loss: 0.0477 - acc: 0.9858 - val_loss: 0.0074 - val_acc: 0.9985
Q 4+584   T 588  ☑ 588 
Q 519+67  T 586  ☑ 586 
Q 444+3   T 447  ☑ 447 
Q 234+23  T 257  ☑ 257 
Q 916+6   T 922  ☑ 922 
Q 71+631  T 702  ☑ 702 
Q 587+1   T 588  ☑ 588 
Q 584+551 T 1135 ☑ 1135
Q 41+14   T 55   ☑ 55  
Q 105+712 T 817  ☑ 817 

--------------------------------------------------
Iteration 45
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [===

45000/45000 [==============================] - 25s 551us/step - loss: 0.0010 - acc: 0.9999 - val_loss: 0.0035 - val_acc: 0.9990
Q 29+621  T 650  ☑ 650 
Q 820+994 T 1814 ☑ 1814
Q 81+630  T 711  ☑ 711 
Q 551+74  T 625  ☑ 625 
Q 21+914  T 935  ☑ 935 
Q 1+954   T 955  ☑ 955 
Q 249+24  T 273  ☑ 273 
Q 35+75   T 110  ☑ 110 
Q 411+3   T 414  ☑ 414 
Q 305+74  T 379  ☑ 379 

--------------------------------------------------
Iteration 58
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 25s 552us/step - loss: 0.0350 - acc: 0.9897 - val_loss: 0.0044 - val_acc: 0.9994
Q 495+868 T 1363 ☑ 1363
Q 6+710   T 716  ☑ 716 
Q 55+908  T 963  ☑ 963 
Q 711+13  T 724  ☑ 724 
Q 41+91   T 132  ☑ 132 
Q 4+592   T 596  ☑ 596 
Q 899+605 T 1504 ☑ 1504
Q 331+100 T 431  ☑ 431 
Q 78+27   T 105  ☑ 105 
Q 51+53   T 104  ☑ 104 

--------------------------------------------------
Iteration 59
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [===

...

45000/45000 [==============================] - 28s 615us/step - loss: 2.6104e-04 - acc: 1.0000 - val_loss: 9.5121e-04 - val_acc: 0.9999
Q 5+691   T 696  ☑ 696 
Q 15+177  T 192  ☑ 192 
Q 4+306   T 310  ☑ 310 
Q 835+53  T 888  ☑ 888 
Q 798+28  T 826  ☑ 826 
Q 58+535  T 593  ☑ 593 
Q 832+371 T 1203 ☑ 1203
Q 49+2    T 51   ☑ 51  
Q 588+115 T 703  ☑ 703 
Q 462+108 T 570  ☑ 570 

--------------------------------------------------
Iteration 142
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 26s 580us/step - loss: 2.2671e-04 - acc: 1.0000 - val_loss: 8.3993e-04 - val_acc: 0.9999
Q 563+555 T 1118 ☑ 1118
Q 9+805   T 814  ☑ 814 
Q 7+453   T 460  ☑ 460 
Q 56+773  T 829  ☑ 829 
Q 8+607   T 615  ☑ 615 
Q 309+235 T 544  ☑ 544 
Q 80+229  T 309  ☑ 309 
Q 699+440 T 1139 ☑ 1139
Q 973+396 T 1369 ☑ 1369
Q 34+108  T 142  ☑ 142 

--------------------------------------------------
Iteration 143
Train on 45000 samples, validate on 5000 samples
Epoch 1/

45000/45000 [==============================] - 25s 551us/step - loss: 1.6912e-04 - acc: 1.0000 - val_loss: 4.6960e-04 - val_acc: 0.9999
Q 177+41  T 218  ☑ 218 
Q 71+98   T 169  ☑ 169 
Q 899+26  T 925  ☑ 925 
Q 530+702 T 1232 ☑ 1232
Q 14+90   T 104  ☑ 104 
Q 0+367   T 367  ☑ 367 
Q 701+512 T 1213 ☑ 1213
Q 313+31  T 344  ☑ 344 
Q 5+625   T 630  ☑ 630 
Q 508+0   T 508  ☑ 508 

--------------------------------------------------
Iteration 156
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 25s 554us/step - loss: 1.4928e-04 - acc: 1.0000 - val_loss: 9.4374e-04 - val_acc: 0.9998
Q 986+79  T 1065 ☑ 1065
Q 266+4   T 270  ☑ 270 
Q 88+89   T 177  ☑ 177 
Q 848+793 T 1641 ☑ 1641
Q 145+616 T 761  ☑ 761 
Q 248+715 T 963  ☑ 963 
Q 940+882 T 1822 ☑ 1822
Q 53+281  T 334  ☑ 334 
Q 657+88  T 745  ☑ 745 
Q 271+6   T 277  ☑ 277 

--------------------------------------------------
Iteration 157
Train on 45000 samples, validate on 5000 samples
Epoch 1/

45000/45000 [==============================] - 25s 550us/step - loss: 0.0262 - acc: 0.9923 - val_loss: 0.0058 - val_acc: 0.9984
Q 335+23  T 358  ☑ 358 
Q 306+868 T 1174 ☑ 1174
Q 193+5   T 198  ☑ 198 
Q 171+77  T 248  ☑ 248 
Q 682+98  T 780  ☑ 780 
Q 221+440 T 661  ☑ 661 
Q 802+26  T 828  ☑ 828 
Q 899+26  T 925  ☑ 925 
Q 26+42   T 68   ☑ 68  
Q 613+4   T 617  ☑ 617 

--------------------------------------------------
Iteration 170
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 25s 550us/step - loss: 0.0012 - acc: 0.9999 - val_loss: 0.0010 - val_acc: 0.9999
Q 556+979 T 1535 ☑ 1535
Q 764+7   T 771  ☑ 771 
Q 5+336   T 341  ☑ 341 
Q 291+14  T 305  ☑ 305 
Q 134+81  T 215  ☑ 215 
Q 32+394  T 426  ☑ 426 
Q 26+31   T 57   ☑ 57  
Q 482+352 T 834  ☑ 834 
Q 8+114   T 122  ☑ 122 
Q 799+132 T 931  ☑ 931 

--------------------------------------------------
Iteration 171
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [=

45000/45000 [==============================] - 25s 561us/step - loss: 6.6526e-05 - acc: 1.0000 - val_loss: 3.2060e-04 - val_acc: 0.9999
Q 45+53   T 98   ☑ 98  
Q 870+282 T 1152 ☑ 1152
Q 934+988 T 1922 ☑ 1922
Q 6+963   T 969  ☑ 969 
Q 234+37  T 271  ☑ 271 
Q 83+744  T 827  ☑ 827 
Q 5+168   T 173  ☑ 173 
Q 92+905  T 997  ☑ 997 
Q 7+712   T 719  ☑ 719 
Q 862+49  T 911  ☑ 911 

--------------------------------------------------
Iteration 184
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 25s 556us/step - loss: 0.0306 - acc: 0.9923 - val_loss: 0.0019 - val_acc: 0.9996
Q 793+8   T 801  ☑ 801 
Q 663+607 T 1270 ☑ 1270
Q 4+349   T 353  ☑ 353 
Q 991+570 T 1561 ☑ 1561
Q 63+745  T 808  ☑ 808 
Q 590+9   T 599  ☑ 599 
Q 278+143 T 421  ☑ 421 
Q 7+732   T 739  ☑ 739 
Q 536+79  T 615  ☑ 615 
Q 334+87  T 421  ☑ 421 

--------------------------------------------------
Iteration 185
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/

45000/45000 [==============================] - 25s 558us/step - loss: 6.2113e-05 - acc: 1.0000 - val_loss: 2.7883e-04 - val_acc: 0.9999
Q 983+611 T 1594 ☑ 1594
Q 443+758 T 1201 ☑ 1201
Q 960+6   T 966  ☑ 966 
Q 35+94   T 129  ☑ 129 
Q 589+527 T 1116 ☑ 1116
Q 177+0   T 177  ☑ 177 
Q 887+782 T 1669 ☑ 1669
Q 427+8   T 435  ☑ 435 
Q 583+63  T 646  ☑ 646 
Q 514+871 T 1385 ☑ 1385

--------------------------------------------------
Iteration 198
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 25s 553us/step - loss: 0.0281 - acc: 0.9924 - val_loss: 0.0030 - val_acc: 0.9993
Q 387+76  T 463  ☑ 463 
Q 149+93  T 242  ☑ 242 
Q 3+18    T 21   ☑ 21  
Q 45+3    T 48   ☑ 48  
Q 852+628 T 1480 ☑ 1480
Q 424+75  T 499  ☑ 499 
Q 18+818  T 836  ☑ 836 
Q 151+396 T 547  ☑ 547 
Q 24+833  T 857  ☑ 857 
Q 55+11   T 66   ☑ 66  

--------------------------------------------------
Iteration 199
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/

In [31]:
model.save('rnn_model_%siter.h5' %no_iter)